In [15]:
# Let's just look at what's actually on the page
def simple_page_analysis(wallet_address):
    url = f"https://jup.ag/portfolio/{wallet_address}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    print("=== PAGE CONTENT ANALYSIS ===")
    print(f"Title: {soup.title.string}")
    print(f"Total page length: {len(response.text)} characters")
    
    # Look for any visible text that might contain portfolio info
    page_text = soup.get_text()
    print(f"Visible text length: {len(page_text)} characters")
    
    # Show first 1000 characters of visible text
    print("\n=== FIRST 1000 CHARACTERS OF VISIBLE TEXT ===")
    print(page_text[:1000])
    
    # Look for specific keywords in the full page source
    keywords = ['balance', 'token', 'price', 'value', 'portfolio', 'SOL', 'USDC']
    print(f"\n=== KEYWORD SEARCH IN PAGE SOURCE ===")
    for keyword in keywords:
        count = response.text.lower().count(keyword.lower())
        print(f"'{keyword}': found {count} times")

# Run the analysis
simple_page_analysis("61rVn8zeoikzFuT2cuXAd3Qv9RxhMDvG52xL4LHLv7Nu")

=== PAGE CONTENT ANALYSIS ===
Title: Jupiter Portfolio: Track every Solana DeFi position | Jupiter
Total page length: 13852 characters
Visible text length: 61 characters

=== FIRST 1000 CHARACTERS OF VISIBLE TEXT ===
Jupiter Portfolio: Track every Solana DeFi position | Jupiter

=== KEYWORD SEARCH IN PAGE SOURCE ===
'balance': found 0 times
'token': found 0 times
'price': found 0 times
'value': found 0 times
'portfolio': found 6 times
'SOL': found 2 times
'USDC': found 0 times


# Solana Portfolio Tracker

This notebook will track Solana wallet portfolios and display trading data in a custom format.

**Target Wallet:** `61rVn8zeoikzFuT2cuXAd3Qv9RxhMDvG52xL4LHLv7Nu`

In [18]:
# Import necessary libraries for web scraping
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import datetime
from bs4 import BeautifulSoup
import time

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("Libraries loaded! Ready to scrape Jupiter portfolio data.")

Libraries loaded! Ready to scrape Jupiter portfolio data.


In [14]:
# Define the wallet address to track
WALLET_ADDRESS = "61rVn8zeoikzFuT2cuXAd3Qv9RxhMDvG52xL4LHLv7Nu"

print(f"Tracking wallet: {WALLET_ADDRESS}")
print(f"Jupiter Portfolio URL: https://jup.ag/portfolio/{WALLET_ADDRESS}")

Tracking wallet: 61rVn8zeoikzFuT2cuXAd3Qv9RxhMDvG52xL4LHLv7Nu
Jupiter Portfolio URL: https://jup.ag/portfolio/61rVn8zeoikzFuT2cuXAd3Qv9RxhMDvG52xL4LHLv7Nu


## Step 1: Research Available APIs

We need to find APIs that can give us:
- Wallet token balances
- Transaction history
- Token prices
- Trading data

Potential APIs:
- Jupiter API
- Solana RPC
- Solscan API
- CoinGecko for prices

In [19]:
# Let's try to scrape the Jupiter portfolio page directly
def scrape_jupiter_portfolio(wallet_address):
    """
    Scrape portfolio data from Jupiter website
    """
    url = f"https://jup.ag/portfolio/{wallet_address}"
    
    # Set headers to mimic a real browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        print(f"Fetching data from: {url}")
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        # Parse the HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Jupiter likely loads data with JavaScript, so we might need to look for JSON data
        # Let's check what we can find in the page source
        print(f"Page title: {soup.title.string if soup.title else 'No title'}")
        print(f"Page content length: {len(response.text)} characters")
        
        # Look for script tags that might contain portfolio data
        scripts = soup.find_all('script')
        print(f"Found {len(scripts)} script tags")
        
        return {
            "url": url,
            "status": "fetched",
            "content_length": len(response.text),
            "scripts_found": len(scripts)
        }
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return {"error": str(e)}

# Test the scraping function
result = scrape_jupiter_portfolio(WALLET_ADDRESS)
print(f"Scraping result: {result}")

Fetching data from: https://jup.ag/portfolio/61rVn8zeoikzFuT2cuXAd3Qv9RxhMDvG52xL4LHLv7Nu
Page title: Jupiter Portfolio: Track every Solana DeFi position | Jupiter
Page content length: 13852 characters
Found 15 script tags
Scraping result: {'url': 'https://jup.ag/portfolio/61rVn8zeoikzFuT2cuXAd3Qv9RxhMDvG52xL4LHLv7Nu', 'status': 'fetched', 'content_length': 13852, 'scripts_found': 15}


## Step 2: Data Collection Functions

We'll create functions to:
1. Fetch token balances
2. Get transaction history
3. Calculate portfolio value
4. Track trading performance

In [17]:
# Alternative approach: Check if Jupiter has a simple API endpoint
def check_jupiter_api():
    """
    Check if Jupiter has any accessible API endpoints
    """
    # Try some common API patterns
    api_urls = [
        f"https://api.jup.ag/portfolio/{WALLET_ADDRESS}",
        f"https://jup.ag/api/portfolio/{WALLET_ADDRESS}",
        f"https://api.jup.ag/v1/portfolio/{WALLET_ADDRESS}"
    ]
    
    for url in api_urls:
        try:
            print(f"Trying API endpoint: {url}")
            response = requests.get(url)
            print(f"Status code: {response.status_code}")
            
            if response.status_code == 200:
                print("✅ Found working API endpoint!")
                data = response.json()
                print(f"Data preview: {str(data)[:200]}...")
                return url, data
            else:
                print(f"❌ Not accessible (status: {response.status_code})")
                
        except Exception as e:
            print(f"❌ Error: {e}")
        
        print("-" * 50)
    
    return None, None

# Test for API endpoints
api_url, api_data = check_jupiter_api()

if api_url:
    print(f"Great! We can use API: {api_url}")
else:
    print("No direct API found. We'll need to use web scraping with browser automation.")

Trying API endpoint: https://api.jup.ag/portfolio/61rVn8zeoikzFuT2cuXAd3Qv9RxhMDvG52xL4LHLv7Nu
Status code: 401
❌ Not accessible (status: 401)
--------------------------------------------------
Trying API endpoint: https://jup.ag/api/portfolio/61rVn8zeoikzFuT2cuXAd3Qv9RxhMDvG52xL4LHLv7Nu
Status code: 404
❌ Not accessible (status: 404)
--------------------------------------------------
Trying API endpoint: https://api.jup.ag/v1/portfolio/61rVn8zeoikzFuT2cuXAd3Qv9RxhMDvG52xL4LHLv7Nu
Status code: 401
❌ Not accessible (status: 401)
--------------------------------------------------
No direct API found. We'll need to use web scraping with browser automation.


## Next Steps - Web Scraping Approach

Since Jupiter loads data dynamically, we have a few options:

### Option 1: Browser Automation (Recommended)
Install Selenium to control a real browser:
```bash
pip install selenium
```

### Option 2: Network Analysis
Inspect Jupiter's network requests to find the actual API calls it makes

### Option 3: Alternative Data Sources
Use other Solana portfolio trackers like:
- Solscan API
- DeBank API
- Direct Solana RPC calls

**Let's start with Option 1 - it's the most reliable for scraping modern web apps!**

In [ ]:
  print("Hello, this is working!")

In [20]:
# SUCCESS! Working Selenium approach to get Jupiter portfolio data
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

def get_jupiter_portfolio_data(wallet_address):
    """
    Get portfolio data from Jupiter using Selenium (with all PnL and values)
    """
    url = f"https://jup.ag/portfolio/{wallet_address}"
    
    # Set up Chrome options  
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in background
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    try:
        print(f"📊 Fetching portfolio data for: {wallet_address}")
        print(f"🔗 URL: {url}")
        
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(url)
        
        # Wait for portfolio data to load
        print("⏳ Waiting for portfolio data to load...")
        time.sleep(10)
        
        # Extract portfolio data
        page_text = driver.find_element(By.TAG_NAME, "body").text
        
        # Find portfolio elements
        portfolio_data = {}
        
        # Look for dollar amounts and portfolio values
        elements = driver.find_elements(By.XPATH, "//*[contains(text(), '$') or contains(text(), 'SOL') or contains(text(), 'PnL')]")
        
        portfolio_values = []
        for element in elements:
            text = element.text.strip()
            if text and '$' in text:
                portfolio_values.append(text)
        
        # Extract specific metrics from visible text
        lines = page_text.split('\n')
        for i, line in enumerate(lines):
            if 'Net Worth' in line and i+1 < len(lines):
                portfolio_data['net_worth'] = lines[i+1]
            elif 'Holdings PnL' in line and i+1 < len(lines):
                portfolio_data['holdings_pnl'] = lines[i+1]
            elif 'JUP Holdings' in line and i+1 < len(lines):
                portfolio_data['jup_holdings'] = lines[i+1]
        
        portfolio_data['all_dollar_values'] = portfolio_values
        portfolio_data['wallet'] = wallet_address
        portfolio_data['timestamp'] = time.strftime('%Y-%m-%d %H:%M:%S')
        
        driver.quit()
        
        print("✅ Portfolio data extracted successfully!")
        return portfolio_data
        
    except Exception as e:
        print(f"❌ Error: {e}")
        try:
            driver.quit()
        except:
            pass
        return None

# Test the function
WALLET_ADDRESS_ACTIVE = "61rVn8zeoikzFuT2cuXAd3Qv9RxhMDvG52xL4LHLv7Nu"
portfolio_data = get_jupiter_portfolio_data(WALLET_ADDRESS_ACTIVE)

if portfolio_data:
    print("\n" + "="*50)
    print("📊 PORTFOLIO SUMMARY")
    print("="*50)
    for key, value in portfolio_data.items():
        print(f"{key}: {value}")
else:
    print("Failed to get portfolio data")

📊 Fetching portfolio data for: 61rVn8zeoikzFuT2cuXAd3Qv9RxhMDvG52xL4LHLv7Nu
🔗 URL: https://jup.ag/portfolio/61rVn8zeoikzFuT2cuXAd3Qv9RxhMDvG52xL4LHLv7Nu
⏳ Waiting for portfolio data to load...
✅ Portfolio data extracted successfully!

📊 PORTFOLIO SUMMARY
net_worth: $0.00
holdings_pnl: +$2,232,512.27
jup_holdings: 0.00
all_dollar_values: ['$0.00', '$2,232,512.27', '$0.00']
wallet: 61rVn8zeoikzFuT2cuXAd3Qv9RxhMDvG52xL4LHLv7Nu
timestamp: 2025-10-04 16:56:32


In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# Your wallet address
wallet_address = "61rVn8zeoikzFuT2cuXAd3Qv9RxhMDvG52xL4LHLv7Nu"
url = f"https://jup.ag/portfolio/{wallet_address}"

# Set up Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")

driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
time.sleep(10)

# Get the data
page_text = driver.find_element(By.TAG_NAME, "body").text
print(page_text[:500])  # First 500 characters

driver.quit()

Swap
Pro
Perps
Lend
Beta
Portfolio
More
Search
Connect
Dashboard
Airdrop Checker
Wallet Manager
Proof of humanity required
Did you know
Jupiter Mobile shows you both holdings & positions natively.
Download the app →
61
61rV...v7Nu
Search for address
⌘K
Net Worth
$0.00
0.00 SOL
Holdings PnL
+$2,232,456.59
New
JUP Holdings
0.00
JUP Staked
0.00%
Positions
Activity
61rV...v7Nu - $0.00
Bug
Feedback
No asset detected
You don’t have any assets in your wallet yet. Once you hold some, they’ll appear here
